# Problem Statement

One of the largest retail chains in the world wants to use their vast data source to build an efficient forecasting model to predict the sales for each SKU in its portfolio at its 76 different stores using historical sales data for the past 3 years on a week-on-week basis. Sales and promotional information is also available for each week - product and store wise.

However, no other information regarding stores and products are available. Can you still forecast accurately the sales values for every such product/SKU-store combination for the next 12 weeks accurately? If yes, then dive right in

# Variable Definition

record_ID: Unique ID for each week store sku combination
week: Starting Date of the week
store_id: Unique ID for each store (no numerical order to be assumed)
sku_id: Unique ID for each product (no numerical order to be assumed)
total_price: Sales Price of the product 
base_price: Base price of the product
is_featured_sku: Was part of the featured item of the week
is_display_sku: Product was on display at a prominent place at the store
units_sold(Target): Total Units sold for that week-store-sku combination

# Approach (high level)

Considered this as a regression problem with 'units_sold' as a target
Generated following new features:
(a) Count of records per 'sku-id','store-id' and combination of both
(b) Average units sold per 'sku-id','store-id' and combination of both
(c) Average base-price & total-price per 'sku-id','store-id' and combination of both
(d) Week of the year
(e) Week number from start of data
(f) Week of the month
(g) Sine & Cosine transform of week number to capture cyclic nature
(e) Price difference percent between base price & total-price
Categorical Encoded 'sku-id' & 'store-id' with MEstimateEncoder()
Trained the data on RandomForest & LGBM Regressor
Tuned the above models

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 

from sklearn.metrics import mean_squared_error,mean_squared_log_error
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV,cross_val_score,RepeatedKFold
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Normalizer,RobustScaler

import xgboost as xgb
import lightgbm as lgb
import sklearn.ensemble as ensemble
import sklearn.metrics as metrics
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from scipy import stats
from scipy.stats import norm, skew


from category_encoders.ordinal import OrdinalEncoder
from category_encoders.woe import WOEEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.sum_coding import SumEncoder
from category_encoders.m_estimate import MEstimateEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.james_stein import JamesSteinEncoder
from category_encoders.one_hot import OneHotEncoder
from scipy.special import boxcox1p
# from bayes_opt import BayesianOptimization

In [7]:
warnings.filterwarnings('ignore')

In [9]:
train=pd.read_csv('/Users/sreevaatsav/Downloads/IOT/Traindata.csv')
test=pd.read_csv('/Users/sreevaatsav/Downloads/IOT/DataSetRetail.csv')
# submission=pd.read_csv('../input/sample_submission_1.csv')

In [10]:
train.head()

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
0,1,17-01-2011,8091,216418,99.0375,111.8625,0,0,20
1,2,17-01-2011,8091,216419,99.0375,99.0375,0,0,28
2,3,17-01-2011,8091,216425,133.9500,133.9500,0,0,19
3,4,17-01-2011,8091,216233,133.9500,133.9500,0,0,44
4,5,17-01-2011,8091,217390,141.0750,141.0750,0,0,52


In [11]:
test.head()

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku
0,212645,16-07-2013,8091,216418,108.3000,108.3000,0,0
1,212646,16-07-2013,8091,216419,109.0125,109.0125,0,0
2,212647,16-07-2013,8091,216425,133.9500,133.9500,0,0
3,212648,16-07-2013,8091,216233,133.9500,133.9500,0,0
4,212649,16-07-2013,8091,217390,176.7000,176.7000,0,0


In [13]:
train.describe()

,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
count,150150.000000,150150.000000,150150.000000,150149.000000,150150.000000,150150.000000,150150.000000,150150.000000
mean,106271.555504,9199.422511,254761.132468,206.626751,219.425927,0.095611,0.133200,51.674206
std,61386.037861,615.591445,85547.306447,103.308516,110.961712,0.294058,0.339792,60.207904
min,1.000000,8023.000000,216233.000000,41.325000,61.275000,0.000000,0.000000,1.000000
25%,53111.250000,8562.000000,217217.000000,130.387500,133.237500,0.000000,0.000000,20.000000
50%,106226.500000,9371.000000,222087.000000,198.075000,205.912500,0.000000,0.000000,35.000000
75%,159452.750000,9731.000000,245338.000000,233.700000,234.412500,0.000000,0.000000,62.000000
max,212644.000000,9984.000000,679023.000000,562.162500,562.162500,1.000000,1.000000,2876.000000


In [14]:
train.dtypes

record_ID            int64
week                object
store_id             int64
sku_id               int64
total_price        float64
base_price         float64
is_featured_sku      int64
is_display_sku       int64
units_sold           int64
dtype: object

In [15]:
train.isna().sum()

record_ID          0
week               0
store_id           0
sku_id             0
total_price        1
base_price         0
is_featured_sku    0
is_display_sku     0
units_sold         0
dtype: int64

In [18]:
train.total_price.mean()

206.62675101066273

In [19]:
#Imputing missing value with the mean
train.total_price=train.total_price.fillna(train.total_price.mean())

In [20]:
print(train.isna().sum().sum())
print(test.isna().sum().sum())

0
0


# Feature Engineering

In [21]:
train.columns

Index(['record_ID', 'week', 'store_id', 'sku_id', 'total_price', 'base_price',
       'is_featured_sku', 'is_display_sku', 'units_sold'],
      dtype='object')

In [23]:
# train["record_ID"]

In [24]:
#New Feature Creation functions

def gen_count_id(train,test,col,name):
    temp=train.groupby(col)['record_ID'].count().reset_index().rename(columns={'record_ID':name})
    train=pd.merge(train,temp,how='left',on=col)
    test=pd.merge(test,temp,how='left',on=col)
    train[name]=train[name].astype(float)
    test[name]=test[name].astype(float)
    train[name].fillna(np.median(temp[name]),inplace=True)
    test[name].fillna(np.median(temp[name]),inplace=True)
    return train,test

def gen_average_units(train,test,col,name):
    temp=train.groupby(col)['units_sold'].mean().reset_index().rename(columns={'units_sold':name})
    train=pd.merge(train,temp,how='left',on=col)
    test=pd.merge(test,temp,how='left',on=col)
    train[name].fillna(np.median(temp[name]),inplace=True)
    test[name].fillna(np.median(temp[name]),inplace=True)
    return train,test

def gen_average_price(train,test,col,price='base_price',name='name'):
    temp=train.groupby(col)[price].mean().reset_index().rename(columns={price:name})
    train=pd.merge(train,temp,how='left',on=col)
    test=pd.merge(test,temp,how='left',on=col)
    train[name].fillna(np.median(temp[name]),inplace=True)
    test[name].fillna(np.median(temp[name]),inplace=True)
    return train,test

In [25]:
train,test = gen_count_id(train,test,col=['sku_id','store_id'],name='count_id_sku_store') #Genearting count of records per 'sku-id & store-id' 
train,test = gen_count_id(train,test,col=['sku_id'],name='count_id_sku') #Genearting count of records per 'sku-id'
train,test = gen_count_id(train,test,col=['store_id'],name='count_id_store') #Genearting count of records per 'store-id'

train,test = gen_average_units(train,test,col=['sku_id','store_id'],name='count_sku_store_id') #Genearting average units sold per 'sku-id & store-id'
train,test = gen_average_units(train,test,col=['store_id'],name='count_store_id') #Genearting average units sold per 'store-id'
train,test = gen_average_units(train,test,col=['sku_id'],name='count_sku_id') #Genearting average units sold per 'sku-id'

train,test = gen_average_price(train,test,col=['sku_id','store_id'],price='base_price',name='price_sku_store') #Genearting average base price per 'sku-id & store-id'
train,test = gen_average_price(train,test,col=['sku_id','store_id'],price='total_price',name='price_to_sku_store') #Genearting average total price per 'sku-id & store-id'
train,test = gen_average_price(train,test,col=['store_id'],price='base_price',name='price_store_id') #Genearting average base price per 'store-id'
train,test = gen_average_price(train,test,col=['sku_id'],price='base_price',name='price_sku_id') #Genearting average base price per 'sku-id'
train,test = gen_average_price(train,test,col=['store_id'],price='total_price',name='price_to_store_id') #Genearting average total price per 'store-id'
train,test = gen_average_price(train,test,col=['sku_id'],price='total_price',name='price_to_sku_id') #Genearting average total price per 'sku-id'

In [26]:
train

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,count_id_sku_store,...,count_id_store,count_sku_store_id,count_store_id,count_sku_id,price_sku_store,price_to_sku_store,price_store_id,price_sku_id,price_to_store_id,price_to_sku_id
0,1,17-01-2011,8091,216418,99.0375,111.8625,0,0,20,130.0,...,1170.0,26.376923,32.805983,88.923869,105.800769,102.117692,181.312372,94.688268,172.272756,91.982702
1,2,17-01-2011,8091,216419,99.0375,99.0375,0,0,28,130.0,...,1170.0,28.307692,32.805983,72.182664,105.839135,102.473942,181.312372,94.908763,172.272756,92.349162
2,3,17-01-2011,8091,216425,133.9500,133.9500,0,0,19,130.0,...,1170.0,25.669231,32.805983,34.019231,131.204135,126.671538,181.312372,128.285210,172.272756,125.156355
3,4,17-01-2011,8091,216233,133.9500,133.9500,0,0,44,130.0,...,1170.0,29.107692,32.805983,46.821206,129.916154,126.167308,181.312372,128.101871,172.272756,124.942208
4,5,17-01-2011,8091,217390,141.0750,141.0750,0,0,52,130.0,...,1170.0,29.469231,32.805983,62.312747,164.439519,152.968269,181.312372,158.990538,172.272756,150.867334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150145,212638,09-07-2013,9984,223245,235.8375,235.8375,0,0,38,130.0,...,2210.0,61.384615,37.853394,68.510537,221.423077,207.693750,197.030107,214.878438,186.580537,203.073612
150146,212639,09-07-2013,9984,223153,235.8375,235.8375,0,0,30,130.0,...,2210.0,53.023077,37.853394,60.407560,226.465385,211.892019,197.030107,220.461485,186.580537,206.303241
150147,212642,09-07-2013,9984,245338,357.6750,483.7875,1,1,31,130.0,...,2210.0,21.553846,37.853394,33.212139,467.838462,425.066538,197.030107,476.750449,186.580537,432.734649
150148,212643,09-07-2013,9984,547934,141.7875,191.6625,0,1,12,130.0,...,2210.0,14.100000,37.853394,21.838213,172.644231,162.954231,197.030107,174.616247,186.580537,166.385369


In [27]:
#Converting week feature
le = OrdinalEncoder()
train['week_1']=le.fit_transform(train['week'])
le = OrdinalEncoder()
test['week_1']=le.fit_transform(test['week'])+130

#Creating week number feature
train['week_num']=train.week_1%52
test['week_num']=test.week_1%52


#Creating feature: percent difference between base price and checkout price.
train['price_diff_percent'] = (train['base_price'] - train['total_price']) / train['base_price']
test['price_diff_percent'] = (test['base_price'] - test['total_price']) / test['base_price']

In [28]:
train.tail()

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,count_id_sku_store,...,count_sku_id,price_sku_store,price_to_sku_store,price_store_id,price_sku_id,price_to_store_id,price_to_sku_id,week_1,week_num,price_diff_percent
150145,212638,09-07-2013,9984,223245,235.8375,235.8375,0,0,38,130.0,...,68.510537,221.423077,207.693750,197.030107,214.878438,186.580537,203.073612,130,26,0.000000
150146,212639,09-07-2013,9984,223153,235.8375,235.8375,0,0,30,130.0,...,60.407560,226.465385,211.892019,197.030107,220.461485,186.580537,206.303241,130,26,0.000000
150147,212642,09-07-2013,9984,245338,357.6750,483.7875,1,1,31,130.0,...,33.212139,467.838462,425.066538,197.030107,476.750449,186.580537,432.734649,130,26,0.260677
150148,212643,09-07-2013,9984,547934,141.7875,191.6625,0,1,12,130.0,...,21.838213,172.644231,162.954231,197.030107,174.616247,186.580537,166.385369,130,26,0.260223
150149,212644,09-07-2013,9984,679023,234.4125,234.4125,0,0,15,130.0,...,16.617094,208.154135,197.033654,197.030107,209.117532,186.580537,198.309455,130,26,0.000000


In [29]:
test.head()

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,count_id_sku_store,count_id_sku,...,count_sku_id,price_sku_store,price_to_sku_store,price_store_id,price_sku_id,price_to_store_id,price_to_sku_id,week_1,week_num,price_diff_percent
0,212645,16-07-2013,8091,216418,108.3000,108.3000,0,0,130.0,8840.0,...,88.923869,105.800769,102.117692,181.312372,94.688268,172.272756,91.982702,131,27,0.0
1,212646,16-07-2013,8091,216419,109.0125,109.0125,0,0,130.0,8710.0,...,72.182664,105.839135,102.473942,181.312372,94.908763,172.272756,92.349162,131,27,0.0
2,212647,16-07-2013,8091,216425,133.9500,133.9500,0,0,130.0,8580.0,...,34.019231,131.204135,126.671538,181.312372,128.285210,172.272756,125.156355,131,27,0.0
3,212648,16-07-2013,8091,216233,133.9500,133.9500,0,0,130.0,9620.0,...,46.821206,129.916154,126.167308,181.312372,128.101871,172.272756,124.942208,131,27,0.0
4,212649,16-07-2013,8091,217390,176.7000,176.7000,0,0,130.0,9100.0,...,62.312747,164.439519,152.968269,181.312372,158.990538,172.272756,150.867334,131,27,0.0


In [30]:
X=train[list(set(train.columns)-set(['record_ID','units_sold','week']))]
Y= np.log1p(train['units_sold'])
X_test=test[list(set(test.columns)-set(['record_ID','week']))]

In [31]:
X.dtypes

sku_id                  int64
base_price            float64
count_id_store        float64
price_to_sku_store    float64
count_id_sku          float64
price_to_sku_id       float64
count_sku_store_id    float64
is_display_sku          int64
week_1                  int64
price_sku_id          float64
is_featured_sku         int64
price_diff_percent    float64
count_store_id        float64
price_sku_store       float64
price_to_store_id     float64
count_id_sku_store    float64
week_num                int64
count_sku_id          float64
price_store_id        float64
store_id                int64
total_price           float64
dtype: object

In [32]:
X['sku_id'] = X['sku_id'].astype('category')
X['store_id'] = X['store_id'].astype('category')

In [33]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150150 entries, 0 to 150149
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   sku_id              150150 non-null  category
 1   base_price          150150 non-null  float64 
 2   count_id_store      150150 non-null  float64 
 3   price_to_sku_store  150150 non-null  float64 
 4   count_id_sku        150150 non-null  float64 
 5   price_to_sku_id     150150 non-null  float64 
 6   count_sku_store_id  150150 non-null  float64 
 7   is_display_sku      150150 non-null  int64   
 8   week_1              150150 non-null  int64   
 9   price_sku_id        150150 non-null  float64 
 10  is_featured_sku     150150 non-null  int64   
 11  price_diff_percent  150150 non-null  float64 
 12  count_store_id      150150 non-null  float64 
 13  price_sku_store     150150 non-null  float64 
 14  price_to_store_id   150150 non-null  float64 
 15  count_id_sku_stor

In [34]:
print(len(X_test.columns))
print(len(X.columns))

21
21


In [35]:
print(X_test.isna().sum().sum())
print(X.isna().sum().sum())

0
0


In [36]:
category_list=['store_id','sku_id']

In [37]:
encoder_final=MEstimateEncoder()
encoder_final.fit(X[category_list], Y)

cat_enc = encoder_final.transform(X[category_list], Y)
continuous_train = X.drop(columns= category_list)
X = pd.concat([cat_enc,continuous_train],axis=1)

test_enc=encoder_final.transform(X_test[category_list])
continuous_test=X_test.drop(columns= category_list)
X_test=pd.concat([test_enc,continuous_test],axis=1)

In [38]:
X.head()

,store_id,sku_id,base_price,count_id_store,price_to_sku_store,count_id_sku,price_to_sku_id,count_sku_store_id,is_display_sku,week_1,...,is_featured_sku,price_diff_percent,count_store_id,price_sku_store,price_to_store_id,count_id_sku_store,week_num,count_sku_id,price_store_id,total_price
0,3.250779,4.201821,111.8625,1170.0,102.117692,8840.0,91.982702,26.376923,0,1,...,0,0.11465,32.805983,105.800769,172.272756,130.0,1,88.923869,181.312372,99.0375
1,3.250779,4.051982,99.0375,1170.0,102.473942,8710.0,92.349162,28.307692,0,1,...,0,0.00000,32.805983,105.839135,172.272756,130.0,1,72.182664,181.312372,99.0375
2,3.250779,3.303344,133.9500,1170.0,126.671538,8580.0,125.156355,25.669231,0,1,...,0,0.00000,32.805983,131.204135,172.272756,130.0,1,34.019231,181.312372,133.9500
3,3.250779,3.715659,133.9500,1170.0,126.167308,9620.0,124.942208,29.107692,0,1,...,0,0.00000,32.805983,129.916154,172.272756,130.0,1,46.821206,181.312372,133.9500
4,3.250779,3.850743,141.0750,1170.0,152.968269,9100.0,150.867334,29.469231,0,1,...,0,0.00000,32.805983,164.439519,172.272756,130.0,1,62.312747,181.312372,141.0750


In [39]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150150 entries, 0 to 150149
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   store_id            150150 non-null  float64
 1   sku_id              150150 non-null  float64
 2   base_price          150150 non-null  float64
 3   count_id_store      150150 non-null  float64
 4   price_to_sku_store  150150 non-null  float64
 5   count_id_sku        150150 non-null  float64
 6   price_to_sku_id     150150 non-null  float64
 7   count_sku_store_id  150150 non-null  float64
 8   is_display_sku      150150 non-null  int64  
 9   week_1              150150 non-null  int64  
 10  price_sku_id        150150 non-null  float64
 11  is_featured_sku     150150 non-null  int64  
 12  price_diff_percent  150150 non-null  float64
 13  count_store_id      150150 non-null  float64
 14  price_sku_store     150150 non-null  float64
 15  price_to_store_id   150150 non-nul

In [40]:
X_test.head()

,store_id,sku_id,base_price,count_id_store,price_to_sku_store,count_id_sku,price_to_sku_id,count_sku_store_id,is_display_sku,week_1,...,is_featured_sku,price_diff_percent,count_store_id,price_sku_store,price_to_store_id,count_id_sku_store,week_num,count_sku_id,price_store_id,total_price
0,3.250779,4.201821,108.3000,1170.0,102.117692,8840.0,91.982702,26.376923,0,131,...,0,0.0,32.805983,105.800769,172.272756,130.0,27,88.923869,181.312372,108.3000
1,3.250779,4.051982,109.0125,1170.0,102.473942,8710.0,92.349162,28.307692,0,131,...,0,0.0,32.805983,105.839135,172.272756,130.0,27,72.182664,181.312372,109.0125
2,3.250779,3.303344,133.9500,1170.0,126.671538,8580.0,125.156355,25.669231,0,131,...,0,0.0,32.805983,131.204135,172.272756,130.0,27,34.019231,181.312372,133.9500
3,3.250779,3.715659,133.9500,1170.0,126.167308,9620.0,124.942208,29.107692,0,131,...,0,0.0,32.805983,129.916154,172.272756,130.0,27,46.821206,181.312372,133.9500
4,3.250779,3.850743,176.7000,1170.0,152.968269,9100.0,150.867334,29.469231,0,131,...,0,0.0,32.805983,164.439519,172.272756,130.0,27,62.312747,181.312372,176.7000


# Model Building

In [41]:
X.columns

Index(['store_id', 'sku_id', 'base_price', 'count_id_store',
       'price_to_sku_store', 'count_id_sku', 'price_to_sku_id',
       'count_sku_store_id', 'is_display_sku', 'week_1', 'price_sku_id',
       'is_featured_sku', 'price_diff_percent', 'count_store_id',
       'price_sku_store', 'price_to_store_id', 'count_id_sku_store',
       'week_num', 'count_sku_id', 'price_store_id', 'total_price'],
      dtype='object')

In [44]:
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size = 0.2,random_state=23)

In [45]:
len(x_train.columns)

21

In [48]:
rf_tuned = RandomForestRegressor(bootstrap=True, criterion='absolute_error', max_depth=30,
                      max_features='sqrt')
rf_tuned.fit(x_train,y_train)

In [ ]:
model_lgb=lgb.LGBMRegressor(bagging_fraction=0.8, bagging_frequency=4, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.5,
              importance_type='split', learning_rate=0.1, max_depth=30,
              min_child_samples=20, min_child_weight=30, min_data_in_leaf=70,
              min_split_gain=0.0001, n_estimators=200, n_jobs=-1,
              num_leaves=1200, objective=None, random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

model_lgb.fit(x_train,y_train)

In [ ]:
prediction_rft_valid=rf_tuned.predict(x_valid)
prediction_lgbmt_valid=model_lgb_tuned.predict(x_valid)

rf_tuned_msle=100*mean_squared_log_error(y_valid,prediction_rft_valid)
lgbm_tuned_msle=100*mean_squared_log_error(y_valid,prediction_lgbmt_valid)

prediction_ensemble_tuned=(((1-rf_tuned_msle)*prediction_rft_valid)+((1-lgbm_tuned_msle)*prediction_lgbmt_valid))/(2-rf_tuned_msle-lgbm_tuned_msle)

ensemble_tuned_msle=100*mean_squared_log_error(y_valid,prediction_ensemble_tuned)


print("RF Base: {}; RF Tuned: {}".format(rf_base_msle,rf_tuned_msle))
print("LGBM Base: {}; LGBM Tuned: {}".format(lgbm_base_msle,lgbm_tuned_msle))
print("Ensemble Base: {}; Ensemble Tuned: {}".format(ensemble_base_msle,ensemble_tuned_msle))

In [ ]:
model = lgb.LGBMRegressor(bagging_fraction=0.8, bagging_frequency=4, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.5,
              importance_type='split', learning_rate=0.1, max_depth=30,
              min_child_samples=20, min_child_weight=30, min_data_in_leaf=70,
              min_split_gain=0.0001, n_estimators=100, n_jobs=-1,
              num_leaves=1400, objective=None, random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

model.fit(X,Y)

In [ ]:
X_test.head()

In [ ]:
del X_test['week_num1']

In [ ]:
prediction=model.predict(X_test)

In [ ]:
final_prediction=np.round(np.expm1(prediction))
submission['units_sold']=final_prediction

In [ ]:
submission.head()

In [ ]:
#submission.to_csv('AV_DemandForecast_05.csv',index=False)